In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from torch import load, FloatTensor, no_grad, device

c:\Users\SalmaneElFtouh\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\SalmaneElFtouh\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\SalmaneElFtouh\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:

device = device("cpu")

model = load('maskrcnn_resnet50_fpn.pth')
model = model.to(device)
model.eval()

print("Good!")

Good!


In [3]:


loaded_image=cv2.imread('sample3.jpg')

image = loaded_image.transpose((2,0,1))
image = image / 255.0

batch_input = [image]
batch_input = FloatTensor(batch_input)
batch_input = batch_input.to(device)

with no_grad():
    outputs = model(batch_input)[0]

labels_list=["__background__",'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', None, 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', None, 'backpack', 'umbrella', None, None, 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', None, 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', None, 'dining table', None, None, 'toilet', None, 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', None, 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
boxes = outputs["boxes"]
masks = outputs["masks"]
labels=outputs["labels"]
scores=outputs["scores"]
maskfill=np.ones(loaded_image.shape,dtype='uint8')*130
print(outputs.keys())

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask ==1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] ,
                                  image[:, :, c])
    return image

for box,label,score,mask in zip(boxes,labels,scores,masks):
    if score>=0.5:
        
        mask=mask[0].cpu().numpy()
        mask=np.where(mask >0.5,1,0)
        mask=np.uint8(mask)
       
        rand_color = random.choices(range(256), k=3)
        xmin, ymin, xmax, ymax = box.cpu().numpy().astype(int)
        cv2.rectangle(loaded_image, (xmin, ymin), (xmax, ymax), rand_color, 2)
        cv2.putText(loaded_image,labels_list[label.tolist()],(xmin, ymin),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        
        
        loaded_image=apply_mask(loaded_image,mask,(3,34,213))
print("show result")
cv2.imshow('image_segmentation',loaded_image)
cv2.waitKey(0)
cv2.destroyAllWindows() 

C:\Users\SalmaneElFtouh\AppData\Local\Temp\ipykernel_21160\3305479921.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  batch_input = FloatTensor(batch_input)


dict_keys(['boxes', 'labels', 'scores', 'masks'])
show result
